In [ ]:
#카테고리 분류
import torch 
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from BERT import BertClassifier, Bert_Tokenizer
    
bert = BertModel.from_pretrained('kykim/bert-kor-base')
model = BertClassifier(bert, 64, 4)
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()

def out_category(task) :
    input = [f"{task}"]

    input_ids, attention_mask = Bert_Tokenizer(input)

    with torch.no_grad() :
        output = model(input_ids[0], attention_mask[0])
        pred_label = F.softmax(output).argmax().item()

    reverse_label_map = {0 : '취미', 1 : '가사', 2 : '모임', 3 : '공부'}

    category = reverse_label_map[pred_label]

    return category

c:\Users\osl93\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\osl93\AppData\Local\Temp\ipykernel_15828\1585104110.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe

In [ ]:
#성공확률 예측
import pandas as pd
import numpy as np
from datetime import datetime
import random
from sklearn.linear_model import LogisticRegression

def generate_normal_hours(size=1, mean=336, std=50, min_val=24, max_val=8760):
    samples = np.random.normal(mean, std, size)
    samples = np.clip(samples, min_val, max_val)
    return int(samples.astype(int))

def generate_biased_labels(size=1, prob=0.9):
    return int(np.random.choice([0, 1], size=size, p=[1 - prob, prob]))

def mean(list) :
    return sum(list) / len(list)

n_sample = 500

hours = []
labels = []

for __ in range(n_sample) :
    hour = generate_normal_hours()
    label = generate_biased_labels()
    hours.append(hour)
    labels.append(label)

pri = ['HIGH', 'MEDIUM', 'LOW']
pris = []

for __ in range(n_sample) :
    n = random.randint(0,2)
    pris.append(pri[n])

cat = ['가사', '취미', '공부', '모임']
cats = []

for __ in range(n_sample) :
    n = random.randint(0,3)
    cats.append(cat[n])

df = pd.DataFrame({'hours' : hours,
              'success' : labels})

categories = pd.get_dummies(cats).astype(int)
priorities = pd.get_dummies(pris).astype(int)

df = pd.concat([categories, priorities, df], axis = 1)

X_train = df.drop(columns=['success'])
y_train = df['success']

logistic = LogisticRegression()

logistic.fit(X_train, y_train)

def get_hours(createdDate, deadline) :
    c = datetime.strptime(createdDate[:-7], "%Y-%m-%dT%H:%M:%S")
    d = datetime.strptime(deadline[:-4], "%Y-%m-%dT%H:%M:%S")
    t = d-c
    return round(t.days*24 + t.seconds/3600, 1)

def add_task(category, priority, createdDate, deadline, status):
    new_row = {'가사': 0, '공부': 0, '모임': 0, '취미': 0, 'HIGH': 0, 'LOW': 0, 'MEDIUM': 0, 'hours': get_hours(createdDate, deadline), 'success': None}
    new_row[category] = 1
    new_row[priority] = 1
    if status == 'COMPLETED' :
        new_row['success'] = 1
    elif status == 'DELAYED' :
        new_row['success'] = 0
    else :
        new_row['success'] = None

    return new_row

C:\Users\osl93\AppData\Local\Temp\ipykernel_15828\3233919529.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(samples.astype(int))
C:\Users\osl93\AppData\Local\Temp\ipykernel_15828\3233919529.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(np.random.choice([0, 1], size=size, p=[1 - prob, prob]))
c:\Users\osl93\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.or

In [ ]:
#서버실행
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/category', methods=['POST'])
def predict_category():
    data = request.json
    task = data['content']
    category = out_category(task)

    return jsonify({"category": category})

@app.route('/train', methods=['POST'])
def model_train():
    data = request.json 
    try :
        task = add_task(data['category'], data['priority'], data['createdDate'], data['deadline'], data['status'])
        train_data = pd.DataFrame([task])

        global X_train
        global y_train

        X_train = pd.concat([X_train, train_data.drop(columns=['success'])], axis=0)
        y_train = pd.concat([y_train, train_data['success']], axis=0)

        logistic.fit(X_train, y_train)

        model_result = "Model trained successfully"

        return jsonify({'current_data': task, 'model':model_result})
    except :
        return jsonify({'error': 'Invalid data format'})

@app.route('/prob', methods=['POST'])
def receive_prob():
    data = request.json

    task = add_task(data['category'], data['priority'], data['createdDate'], data['deadline'], data['status'])
    X_data = pd.DataFrame([task]).drop(columns=['success'])
    
    prob = (f'{logistic.predict_proba(X_data)[0][1] * 100:.0f}%')

    return jsonify({'probability':prob})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port =5001, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.219.113:5001
Press CTRL+C to quit
